# Introduction

This notebook shows an example of relaxing the LiFePO4 crystal.

In [1]:
import numpy as np

from pymatgen.ext.matproj import MPRester
from pymatgen.util.coord import pbc_diff

from m3gnet.models import Relaxer

In [2]:
mpr = MPRester()
lfp = mpr.get_structure_by_material_id("mp-19017")  # This is LiFePO4.

lfp_strained = lfp.copy()  # We create a copy.
strains = np.random.uniform(low=-0.05, high=0.05, size=3)  # Create a random strain between -5% and 5% for each direction
lfp_strained.apply_strain(strains)
lfp_strained.perturb(0.1)  # In addition to the lattice strains, we also perturb the atoms by a distance of 0.1 angstrom.

In [3]:
relaxer = Relaxer()
%time  relax_results = relaxer.relax(lfp_strained)
relaxed = relax_results['final_structure']

/Users/shyue/miniconda3/envs/mavrl/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("PartitionedCall:4", shape=(None,), dtype=int32), values=Tensor("PartitionedCall:3", shape=(None, 3, 3), dtype=float32), dense_shape=Tensor("PartitionedCall:5", shape=(3,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/Users/shyue/miniconda3/envs/mavrl/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("PartitionedCall:1", shape=(2628,), dtype=int32), values=Tensor("Neg:0", shape=(2628, 3), dtype=float32), dense_shape=Tensor("PartitionedCall:2", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
2022-05-31 15:12:30.665157: W tensorflow/core/platform/profile_utils/cp

      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 15:12:30     -188.051178*      35.3037


/Users/shyue/miniconda3/envs/mavrl/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradients/m3g_net/three_d_interaction_2/GatherV2_1_grad/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradients/m3g_net/three_d_interaction_2/GatherV2_1_grad/Reshape:0", dtype=float32), dense_shape=Tensor("gradients/m3g_net/three_d_interaction_2/GatherV2_1_grad/Cast:0", shape=(None,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/Users/shyue/miniconda3/envs/mavrl/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradients/m3g_net/graph_network_layer_2/gated_atom_update_2/GatherV2_grad/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradients/m3g_net/graph_network_layer_2/gated_atom_update_2/GatherV2_grad/Reshape

FIRE:    1 15:12:32     -181.340408*      96.6008
FIRE:    2 15:12:32     -188.431458*      51.0341
FIRE:    3 15:12:32     -189.647415*       5.0530
FIRE:    4 15:12:32     -189.025314*      38.3522
FIRE:    5 15:12:33     -189.307358*      31.2309
FIRE:    6 15:12:33     -189.671722*      17.7556
FIRE:    7 15:12:33     -189.856201*       3.1567
FIRE:    8 15:12:33     -189.775055*      15.3738
FIRE:    9 15:12:33     -189.790939*      14.5070
FIRE:   10 15:12:33     -189.819687*      12.8315
FIRE:   11 15:12:33     -189.856003*      10.4586
FIRE:   12 15:12:33     -189.893524*       7.5417
FIRE:   13 15:12:33     -189.926361*       4.2685
FIRE:   14 15:12:33     -189.950577*       2.9155
FIRE:   15 15:12:33     -189.965820*       2.8362
FIRE:   16 15:12:34     -189.976242*       5.1841
FIRE:   17 15:12:34     -189.977158*       5.1106
FIRE:   18 15:12:34     -189.978989*       4.9645
FIRE:   19 15:12:34     -189.981644*       4.7485
FIRE:   20 15:12:34     -189.985092*       4.4661


Note that the relaxation only took < 20s.

In [4]:
print(f"Original lattice parameters are {lfp.lattice.abc}")
print(f"Strained lattice parameters are {lfp_strained.lattice.abc}")
print(f"Relaxed lattice parameters are {relaxed.lattice.abc}")

Original lattice parameters are (4.746441000202679, 10.443730003189714, 6.090226001838521)
Strained lattice parameters are (4.750053179813627, 10.356606838609148, 5.997110481663704)
Relaxed lattice parameters are (4.747618545215528, 10.481805968108853, 6.107802122731116)


In [5]:
print(f"Diff in fractional coords:\n{pbc_diff(lfp.frac_coords, relaxed.frac_coords)}")

Diff in fractional coords:
[[-3.92604728e-03  4.07413997e-03  5.80356625e-03]
 [ 1.43250511e-02  3.27431932e-03  4.42699735e-03]
 [-3.52458018e-03 -3.08654170e-03  1.07293845e-02]
 [-4.26860840e-03  4.58356758e-03 -2.96369922e-03]
 [-5.55072174e-03  1.54835105e-03  2.47354774e-03]
 [ 3.80921860e-03  4.32305748e-03  5.12550707e-03]
 [ 3.53915883e-03 -8.38400924e-04  9.06917571e-03]
 [ 2.47973569e-03 -8.64836750e-04  3.59526770e-03]
 [-7.26667302e-05 -1.35534223e-03 -2.53138511e-03]
 [ 1.14946368e-03 -4.82260909e-05  7.54579410e-03]
 [ 4.24883341e-03  6.67052939e-04  3.70532881e-03]
 [ 9.17448738e-04  3.24917845e-03  3.69315336e-04]
 [ 5.82139415e-04 -3.18889133e-03 -3.03101342e-03]
 [ 2.23741485e-03  2.38577303e-03 -3.57929180e-04]
 [ 3.79457349e-03 -3.22175818e-04  1.67082825e-03]
 [ 2.20562748e-04  5.94968752e-03 -1.96480681e-03]
 [ 5.03072366e-03  5.13620895e-04 -6.28382551e-04]
 [ 5.79797481e-03  3.02933989e-03 -2.87900284e-03]
 [-5.31171473e-03 -6.63377559e-04  2.84482729e-03]
 [-2

Quite clealy, the relaxation using the M3GNet universal IAP has brought the lattice parameters much closer to the original DFT one and the coordinates are also within $10^{-3}$ of the original fractional coordinates.